# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=transformerS",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [3]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [256] # This is the sequence length

cfg.case.server.has_external_data = True

### Instantiate all parties

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Model architecture transformerS loaded with 53,091,409 parameters and 2,560,000 buffers.
Overall this is a data ratio of  207388:1 for target shape [1, 256] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 1

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: transformerS
        model state: default
        public buffers:

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Computing feature distribution before the linear1 layer from external data.
Feature mean is -1.186221718788147, feature std is 0.8130536675453186.


In [6]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, it was part of Little Rock's first military installation. Since its decommissioning, The Tower Building has housed two museums. It was home to the Arkansas Museum of Natural History and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the headquarters of the Little Rock Æsthetic Club since 1894. 
 The building receives its name from its distinct octagonal tower. Besides being the last remaining structure of the original Little Rock Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme commander of US forces in the South Pacific during World War II. It was also the starting place of the Camden Expedition. In 2011 it was named as one of the top 10 attractions in the s

# Reconstruct user data

In [7]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[12, 13, 13, 29, 31, 50, 82, 198, 198, 220, 220, 228, 257, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 278, 284, 284, 284, 284, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 287, 287, 287, 287, 287, 287, 287, 290, 290, 290, 290, 301, 318, 319, 340, 340, 340, 355, 355, 373, 373, 373, 373, 373, 373, 379, 383, 383, 383, 416, 422, 422, 468, 468, 471, 508, 508, 511, 530, 530, 547, 554, 587, 632, 632, 632, 635, 635, 635, 635, 636, 663, 663, 663, 705, 717, 734, 764, 764, 764, 764, 764, 764, 764, 764, 764, 832, 835, 837, 837, 837, 837, 837, 837, 837, 838, 852, 867, 871, 938, 959, 1141, 1181, 1181, 1201, 1201, 1204, 1248, 1279, 1294, 1295, 1353, 1363, 1438, 1810, 1900, 2159, 2422, 2488, 2520, 2581, 2615, 2615, 2627, 2656, 2813, 2873, 2882, 2954, 3250, 3386, 3399, 3411, 3599, 3611, 3700, 3706, 4318, 4619, 4631, 4631, 4631, 4631, 4631, 4645, 4920, 5140, 5637, 5824, 5878, 6184, 6289, 6427, 6

In [8]:
reconstructed_user_data, stats = attacker.reconstruct2([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[12, 13, 13, 29, 31, 50, 82, 198, 198, 220, 220, 228, 257, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 278, 284, 284, 284, 284, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 287, 287, 287, 287, 287, 287, 287, 290, 290, 290, 290, 301, 318, 319, 340, 340, 340, 355, 355, 373, 373, 373, 373, 373, 373, 379, 383, 383, 383, 416, 422, 422, 468, 468, 471, 508, 508, 511, 530, 530, 547, 554, 587, 632, 632, 632, 635, 635, 635, 635, 636, 663, 663, 663, 705, 717, 734, 764, 764, 764, 764, 764, 764, 764, 764, 764, 832, 835, 837, 837, 837, 837, 837, 837, 837, 838, 852, 867, 871, 938, 959, 1141, 1181, 1181, 1201, 1201, 1204, 1248, 1279, 1294, 1295, 1353, 1363, 1438, 1810, 1900, 2159, 2422, 2488, 2520, 2581, 2615, 2615, 2627, 2656, 2813, 2873, 2882, 2954, 3250, 3386, 3399, 3411, 3599, 3611, 3700, 3706, 4318, 4619, 4631, 4631, 4631, 4631, 4631, 4645, 4920, 5140, 5637, 5824, 5878, 6184, 6289, 6427, 6

In [30]:
reconstructed_user_data, stats = attacker.reconstruct3([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[12, 13, 13, 29, 31, 50, 82, 198, 198, 220, 220, 228, 257, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 262, 278, 284, 284, 284, 284, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 287, 287, 287, 287, 287, 287, 287, 290, 290, 290, 290, 301, 318, 319, 340, 340, 340, 355, 355, 373, 373, 373, 373, 373, 373, 379, 383, 383, 383, 416, 422, 422, 468, 468, 471, 508, 508, 511, 530, 530, 547, 554, 587, 632, 632, 632, 635, 635, 635, 635, 636, 663, 663, 663, 705, 717, 734, 764, 764, 764, 764, 764, 764, 764, 764, 764, 832, 835, 837, 837, 837, 837, 837, 837, 837, 838, 852, 867, 871, 938, 959, 1141, 1181, 1181, 1201, 1201, 1204, 1248, 1279, 1294, 1295, 1353, 1363, 1438, 1810, 1900, 2159, 2422, 2488, 2520, 2581, 2615, 2615, 2627, 2656, 2813, 2873, 2882, 2954, 3250, 3386, 3399, 3411, 3599, 3611, 3700, 3706, 4318, 4619, 4631, 4631, 4631, 4631, 4631, 4645, 4920, 5140, 5637, 5824, 5878, 6184, 6289, 6427, 6

tensor(2.4124) tensor(0) tensor(70)
tensor(2.2988) tensor(0) tensor(119)
tensor(2.4661) tensor(0) tensor(12)
tensor(2.3310) tensor(0) tensor(168)
tensor(2.2849) tensor(0) tensor(33)
tensor(2.3917) tensor(0) tensor(112)
tensor(2.4109) tensor(0) tensor(135)
tensor(2.3193) tensor(0) tensor(66)
tensor(2.3972) tensor(0) tensor(40)
tensor(1043.8137) tensor(149) tensor(103)
tensor(2.4207) tensor(0) tensor(65)
tensor(2.3650) tensor(0) tensor(190)
tensor(2.4185) tensor(0) tensor(124)
tensor(54.4233) tensor(104) tensor(139)
tensor(2.5056) tensor(0) tensor(62)
 Arkansas
 Arkansas94 Rock. of remaining as,,s of fears been building constructed in of Arkansas were Arkansas Ining,,, � distinct became also. War. of structure frontier attractions the first It receives tower the the the the the also is the also Arkansas the since and two forces. being home many housed. as Building Building on II museums Rock Expedition Natural World History from buildings
 the it Rock place  state of state and 2011 the R

tensor(202)